In [ ]:
%%writefile vector_add.ptx
//
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-34385749
// Cuda compilation tools, release 12.5, V12.5.82
// Based on NVVM 7.0.1
//

.version 8.4
.target sm_52
.address_size 64

	// .globl	_Z9vectorAddPKfS0_Pfi

.visible .entry _Z9vectorAddPKfS0_Pfi(
	.param .u64 _Z9vectorAddPKfS0_Pfi_param_0,
	.param .u64 _Z9vectorAddPKfS0_Pfi_param_1,
	.param .u64 _Z9vectorAddPKfS0_Pfi_param_2,
	.param .u32 _Z9vectorAddPKfS0_Pfi_param_3
)
{
	.reg .pred 	%p<2>;
	.reg .f32 	%f<4>;
	.reg .b32 	%r<6>;
	.reg .b64 	%rd<11>;


	ld.param.u64 	%rd1, [_Z9vectorAddPKfS0_Pfi_param_0];
	ld.param.u64 	%rd2, [_Z9vectorAddPKfS0_Pfi_param_1];
	ld.param.u64 	%rd3, [_Z9vectorAddPKfS0_Pfi_param_2];
	ld.param.u32 	%r2, [_Z9vectorAddPKfS0_Pfi_param_3];
	mov.u32 	%r3, %ctaid.x;
	mov.u32 	%r4, %ntid.x;
	mov.u32 	%r5, %tid.x;
	mad.lo.s32 	%r1, %r3, %r4, %r5;
	setp.ge.s32 	%p1, %r1, %r2;
	@%p1 bra 	$L__BB0_2;

	cvta.to.global.u64 	%rd4, %rd1;
	mul.wide.s32 	%rd5, %r1, 4;
	add.s64 	%rd6, %rd4, %rd5;
	cvta.to.global.u64 	%rd7, %rd2;
	add.s64 	%rd8, %rd7, %rd5;
	ld.global.f32 	%f1, [%rd8];
	ld.global.f32 	%f2, [%rd6];
	add.f32 	%f3, %f2, %f1;
	cvta.to.global.u64 	%rd9, %rd3;
	add.s64 	%rd10, %rd9, %rd5;
	st.global.f32 	[%rd10], %f3;

$L__BB0_2:
	ret;

}

Writing vector_add.ptx


In [ ]:
%%writefile run_ptx.c
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_CUDA(call) \
    do { \
        CUresult err = call; \
        if (err != CUDA_SUCCESS) { \
            const char *errStr; \
            cuGetErrorString(err, &errStr); \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, errStr); \
            exit(1); \
        } \
    } while(0)

int main() {
    // Initialize CUDA
    CHECK_CUDA(cuInit(0));

    CUdevice device;
    CUcontext context;
    CHECK_CUDA(cuDeviceGet(&device, 0));
    CHECK_CUDA(cuCtxCreate(&context, 0, device));

    // Load PTX module from file
    CUmodule module;
    CHECK_CUDA(cuModuleLoad(&module, "vector_add.ptx"));

    // Get function
    CUfunction kernel;
    CHECK_CUDA(cuModuleGetFunction(&kernel, module, "_Z9vectorAddPKfS0_Pfi"));

    // Setup data
    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);

    // Initialize arrays
    for (int i = 0; i < n; i++) {
        h_a[i] = (float)i;
        h_b[i] = (float)i * 2.0f;
    }

    // Allocate device memory
    CUdeviceptr d_a, d_b, d_c;
    CHECK_CUDA(cuMemAlloc(&d_a, size));
    CHECK_CUDA(cuMemAlloc(&d_b, size));
    CHECK_CUDA(cuMemAlloc(&d_c, size));

    // Copy to device
    CHECK_CUDA(cuMemcpyHtoD(d_a, h_a, size));
    CHECK_CUDA(cuMemcpyHtoD(d_b, h_b, size));

    // Launch kernel
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;

    void *args[] = { &d_a, &d_b, &d_c, &n };

    CHECK_CUDA(cuLaunchKernel(
        kernel,
        gridSize, 1, 1,    // grid
        blockSize, 1, 1,   // block
        0, NULL,           // shared mem, stream
        args, NULL
    ));

    // Copy result back
    CHECK_CUDA(cuMemcpyDtoH(h_c, d_c, size));

    // Verify
    printf("First 10 results:\n");
    for (int i = 0; i < 10; i++) {
        printf("%.2f + %.2f = %.2f\n", h_a[i], h_b[i], h_c[i]);
    }

    // Cleanup
    cuMemFree(d_a);
    cuMemFree(d_b);
    cuMemFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    cuCtxDestroy(context);

    return 0;
}

Writing run_ptx.c


In [ ]:
!nvcc -o run_ptx run_ptx.c -lcuda
!./run_ptx

First 10 results:
0.00 + 0.00 = 0.00
1.00 + 2.00 = 3.00
2.00 + 4.00 = 6.00
3.00 + 6.00 = 9.00
4.00 + 8.00 = 12.00
5.00 + 10.00 = 15.00
6.00 + 12.00 = 18.00
7.00 + 14.00 = 21.00
8.00 + 16.00 = 24.00
9.00 + 18.00 = 27.00
